# A summary of the numerical calculations for gravitational solitons and its results from incresingly relativistic parameters

#### Authors: Xavier Naranjo, Jacob Herman, Seth Winchell | supervision/theory from Dr.'s S.S. Chabysheva and J.R. Hiller

#### August 2024 | project dates from May to August 2024

## Overview:

This notebook covers the computational methods and results for the project exploring how current interpretations of massive particles under its own quantum mechanical and gravitational influence behave with (ultra)relativistic parameters.

The numerical calculations and theory interpret a particle both quantum mechanically and classically (aka with gravity), taking advantage of spherical symmetry. The particle's location is represented as a quantum mechanical mass distribution/wave function using the Klein-Gordon equation, which takes into account a spherically symmetric metric and is a relativistic representation of the Schrodinger equation. A scaled binding energy of $\epsilon$ is extracted from that equation and is used to solved the metric equations for the particle, represented as two first order differential equations - one for time ($g_{00}$) and one for radius ($g_{rr}$). The resulting metrics are put back into the Klein-Gordon equation for an adjusted resulting $\epsilon$ value and the process is repeated until it converges to a desirable tolerance.

The main independent parameter being studied is the ratio of the Bohr radius ($a$) and Schwarchild radius ($r_s$), defined as $\zeta_{s}={r_s}/a=2G^{2}m^{4}$. As this ratio approaches and passes 1, the relativistic self-gravitation of the particle becomes more important to consider as the wave function is altered by the curved metric.

## 1. Methodology

Two sets of Python code were developed with the same functions in mind in order to verify and help troubleshoot the process - the code sets are similar in structure and methods from the collaborative nature of the project, where smaller differences are explained as needed. The functions used in data generation only involve the ``numpy`` library for data manipulation. All calculations are made with a rescaled unit system to ensure values are within floating point computer precision: $\bar h=1$, $c=1$, $G=6.7 \times 10^{-39} GeV^{-2}$.

Current working scripts for data generation and analysis:
1. ``grav_soliton/scripts/kg_and_metric_functions.py`` (Seth) and ``grav_soliton/scripts/X/Functions`` (Xavier): defines functions for the program between Klein-Gordon Solver, Metric ODEs, and iteration between them for a single $\zeta_s$.
2. ``grav_soliton/scripts/params_to_csv.py`` (Seth) and ``grav_soliton/scripts/X/Integrating_inside_out.py`` (Xavier): Imports the functions from (1) and iterates through $\zeta_s$ values and generates the .csv data files.
3. ``grav_soliton/scripts/csv_to_plot.py``: uses ``matplotlib`` to plot values of $\epsilon$, $A_0$, $\tilde R$, A, B, E/m, and $\bar u$. Allows for multiple plot lines (for different $\zeta_s$).
4. ``grav_soliton/scripts/find_critical_zeta_s.py``: uses bisection method to approximate a critical $\zeta_s$ value. takes the main iteration function in (1) and determines if a $\zeta_s$ value is past the critical value by looking for NaN and overflow during intermediate steps.

For data at a desired $\zeta_s$ value, a set of grid points need to be made to perform calculations at, defined as $N$ intervals in a $\zeta_{max}=r_{max}/a$ grid size, where the grid steps are $\Delta = \zeta_{max}/(N+1)$. The resulting algorithm flow is as follows:

1. **Generate initial data** by either reading existing data or using a fixed guess for the metric and wave function at nonrelativistic conditions ($\zeta_s = 0$).
2. Use values for the metric, expressed as $g_{00}=e^{2A}$ and $g_{rr}=e^{2B}$, as inputs for the **Klein-Gordon equation**. Then, solve the eigenvector-eigenvalue problem associated with the equation to get the **rescaled energy ($\epsilon$) and modified radial wave function, defined as $\bar u(\zeta)$**.
3. Use $\epsilon$ and $\bar u$ from the previous step to solve the two **ODEs that come from Einstein's equations**. **Solve for $A$ and $B$**, the exponential factors for the $g_{00}$ and $g_{rr}$ metric components.
4. **Repeat (loop) steps 2 and 3** to get improved $\epsilon$, $\bar u$, $g_{00}$, and $g_{rr}$ values until $\epsilon$ converges to a specified tolerance.
5. Steps 2-4 apply for a single $\zeta_s$ value; a $\zeta_s$ value too far from 0 (>0.1) needs intermediate $\zeta_s$ values to loop though so that the incoming approximations are not too far off for a non-converging algorithm. For example, if data at $\zeta_s = 0.7$ is needed, step 1 will be done once for $\zeta_s = 0$ and the results from step 4 will be used as inputs for step 2 (another loop), this time with a slightly higher $\zeta_s$ like 0.05, and **$\zeta_s$ will be incremented slightly (repeat steps 2-4)** until $\zeta_s = 0.7$ is reached. The more intermediate steps of $\zeta_s$ are taken, the more accurate the calculations become.

### 1.1. Initial data with fixed metric

Since the Klein-Gordon equation needs input arrays for A and B (exponential factors for $g_{00}$ and $g_{rr}$) before it can iterate between itself and the metric ODEs, it needs to read in an approximate AB data set from either a ``.csv`` file (done easily with ``pandas``) or by a generated metric. The scripts use the metric definition of a solid mass boundary from https://arxiv.org/abs/1711.00735 to create an initial guess for the metric, for low $\zeta_s$ values. Outside the mass boundary $r_0$ for this type of metric, the Schwarzchild definition is used:

\begin{equation}
    g_{00} = 1 - \frac{\zeta_s}{\zeta}, g_{rr} = \frac{1}{g_{00}}
\end{equation}

And for inside the hard mass boundary:

\begin{equation}
    g_{00} = \frac{1}{4}[3\sqrt{f(r_0)} - \sqrt{f(r)}]^2, g_{rr} = 1/f(r)
\end{equation}

where $f(r) = 1 - \frac{r_s}{r_s^3}r^2$. These functions are named ``find_fixed_metric()`` and are rescaled to use $\zeta$ instead of $r$.

### 1.2. Klein-Gordon Equation

The Klein-Gordon equation used for the calculations is simplified to use spherically symmetric spacetimes, the rescaled radial equation $\bar u$, and the ground state angular momentum quantum number $l=0$. This turns it into a matrix-eigenvalue problem from the equation:

\begin{equation}
    -\sqrt{\frac{g_{00}}{g_{rr}}}\frac{d^2}{d\zeta^2}[\sqrt{\frac{g_{00}}{g_{rr}}}\bar u]+\frac{g_{00}}{g_{rr}}\frac{\tilde h^{\prime \prime}}{\tilde h}\bar u + \frac{2}{\zeta_s}g_{00}[1-\frac{1}{g_{00}}]\bar u = \lambda \bar u
\end{equation}

where

\begin{equation}
    \frac{\tilde h^{\prime \prime}}{\tilde h} = \sqrt{\sqrt{\frac{g_{00}}{g_{rr}}}}, \hspace{0.5cm} g_{00}[1-1/g_{00}] \approx 2{e^A}sinh(A)
\end{equation}

The second derivative term is calculated using a finite differences approach, which results in a tridiagonal matrix. The diagonal and off-diagonal terms, defined as $C, D, E$ respectively, simplify out to be:

\begin{equation}
    C=\frac{g_{00}(n)}{g_{rr}(n)}\frac{\tilde h^{\prime \prime}}{\tilde h}+(\frac{4}{\zeta_s}e^{A}sinh(A))+\frac{2g_{00}(n)}{g_{rr}(n)\Delta^2}
\end{equation}

\begin{equation}
    D=-\sqrt{\frac{g_{00}(n)g_{00}(n+1)}{g_{rr}(n)g_{rr}(n+1)}}\frac{1}{\Delta^2} \hspace{0.5cm} E=-\sqrt{\frac{g_{00}(n)g_{00}(n-1)}{g_{rr}(n)g_{rr}(n-1)}}\frac{1}{\Delta^2}
\end{equation}

for an nth point in the linear $\zeta$ grid. After the matrix is constructed then $\lambda$ values found with ``numpy.linalg.eig()`` and the minimum one is taken to correspond to the ground state $\epsilon$ with the equation $\epsilon = \lambda/(1+\sqrt{1+\zeta_s\lambda/2})$. Below is the section of the script that performs the mentioned steps:

In [ ]:
# taken from kg_and_metric_functions.py, not intended to work in this notebook

def kg_find_matrix_const(g00, grr, zeta_vals, zeta_s, zeta_max):
    """
    Creates arrays for matrix values in the tridiagonal
    matrix used in the Klein Gordon equation.

    Parameters:
        g00: 1D np array
        grr: 1D np array
        zeta_vals: 1D np array
        zeta_s: np scalar
        zeta_max: np scalar
    Outputs:
        C: 1D np array
        D: 1D np array
        E: 1D np array
    """
    N_max = len(zeta_vals) #Building constants needed
    DEL = zeta_max/len(zeta_vals)
    
    A, B = metric_to_AB(g00, grr)
    C = np.zeros_like(zeta_vals) #Initialize zero vectors for our constants
    D = np.zeros_like(zeta_vals)
    E = np.zeros_like(zeta_vals)
    for i in range(N_max):
        if i > 0 and i < N_max-1:
            if i > 0:
                D[i] = -np.sqrt(g00[i]/grr[i]) * np.sqrt(g00[i-1]/grr[i-1])*(1/DEL**2) #Constants in the lower diagonal in the matrix
            if i < N_max-1:
                E[i] = -np.sqrt(g00[i]/grr[i]) * np.sqrt(g00[i+1]/grr[i+1])*(1/DEL**2) #Constants in the upper diagonal in the matrix
            H_ratio = (zeta_vals[i+1]*np.sqrt(np.sqrt(g00[i+1]/grr[i+1])) - 2*zeta_vals[i]*np.sqrt(np.sqrt(g00[i]/grr[i])) + zeta_vals[i-1]*np.sqrt(np.sqrt(g00[i-1]/grr[i-1])))/(zeta_vals[i]*np.sqrt(np.sqrt(g00[i]/grr[i]))*DEL**2)
        else:
            H_ratio = 0
        C[i] = (g00[i]/grr[i])*H_ratio + (4/zeta_s) * (np.exp(A[i])*np.sinh(A[i])) + (2/(DEL**2))*(g00[i]/grr[i]) #Constants in the diagonal of the matrix
    return C, D, E

def kg_build_matrix(C, D, E, zeta_vals):
    """
    Creates a 2d matrix of tridiagonal elements from kg_find_matrix_const(),
    to be used in the Klein Gordon equation for epsilon and u.

    Parameters:
        C: 1D np array
        D: 1D np array
        E: 1D np array
        zeta_vals: 1D np array
    Outputs:
        Matrix: 2D np array
    """
    N_max = len(zeta_vals)

    matrix = np.zeros((N_max, N_max)) #Initializes zero matrix
    for i in range(N_max): #Places constants in the correct parts of the matrix
        matrix[i, i] = C[i]
        if i > 0: matrix[i, i-1] = D[i]
        if i < N_max-1: matrix[i, i+1] = E[i]

    return matrix

def kg_solver(g00, grr, zeta_s, zeta_vals, zeta_max):
    """
    Uses a finite difference approach to calculate our u_bar array and our epsilon value

    Parameters:
        g00: 1D np array
        grr: 1D np array
        zeta_s: np scalar
        zeta_vals: 1D np array
        zeta_max: np scalar
    Outputs:
        u_bar: 1D np array
        epsilon: np scalar
    """
    DEL = (zeta_max)/(len(zeta_vals))  # Spacing of intervals
    N_max = len(zeta_vals)
    
    A, B = metric_to_AB(g00, grr)
    C, D, E = kg_find_matrix_const(g00, grr, zeta_vals, zeta_s, zeta_max)
    matrix = kg_build_matrix(C, D, E, zeta_vals) #Builds matrix that is dependent on the A and B vectors we got before
    e_vals, e_vecs = np.linalg.eig(matrix) #Grabs eigenvalues and eigenvectors
    epsilons = e_vals/(1+np.sqrt(1+zeta_s*e_vals/2)) #Calculates the epsilon energies for all the eigenvalues
    N = np.argmin(epsilons) #Finds the index for the smallest epsilon, which is our 0 state energy
    u_bar = e_vecs[:, N] #Gets the eigenvector that corresponds to the 0 energy state
    epsilon = epsilons[N] #Grabs minimum energy

    u_tilde = np.sqrt(g00/grr)*u_bar #Converts into U_tilde
    u_tilde[0] = 0
    u_tilde[N_max-1] = 0
    norm = np.sum(grr*u_tilde**2/np.sqrt(g00)) #Normalizes U_tilde
    u_tilde /= np.sqrt(norm*DEL)

    u_bar = np.sqrt(grr/g00)*u_tilde #Converts back to 

    return u_bar, epsilon

### 1.3. Metric Differential Equations

The GR equations in dimensionless form require $\epsilon$ and $\tilde R$ values that are found from the previous step. These equations are differential:

\begin{equation}
    \frac{dA}{d\zeta}=\frac{1}{2\zeta}(e^{2B}-1)-\frac{\zeta_s\zeta}{4}e^{2B}{\tilde R}^2+\frac{{\zeta_s}^2\zeta}{8}(\frac{d\tilde R}{d\zeta})^2+\frac{\zeta_s\zeta}{4}[1+\frac{1}{2}\zeta_s\epsilon]^{2}e^{2(B-A)}{\tilde R}^2
\end{equation}

\begin{equation}
    \frac{dB}{d\zeta}=-\frac{1}{2\zeta}(e^{2B}-1)+\frac{\zeta_s\zeta}{4}e^{2B}{\tilde R}^2+\frac{{\zeta_s}^2\zeta}{8}(\frac{d\tilde R}{d\zeta})^2+\frac{\zeta_s\zeta}{4}[1+\frac{1}{2}\zeta_s\epsilon]^{2}e^{2(B-A)}{\tilde R}^2
\end{equation}

where $\tilde R={\tilde u}/{\tilde h}. 


## 2. Results and Data

### 2.1. $\bar u$ and metric exponents vs. $\zeta$

As was expected, greater $\zeta_s$ values result in a more "pushed in" wave function, with greater peaks and smaller expectation values. Nonetheless there is no indication just from the $\bar u$ graphs that an asymptotic $\zeta_s$ value exists.

![$\bar u$ values](images/v1_1500_20/u_bar_vs_z.png)

###### $\bar u$ values for six different $\zeta_s$ parameters. Taken with version 1 code (Seth) with 1500 intervals and $\zeta_{max}=20$.

Graphs of the metric are more indicative of what happens as the critical $\zeta_s$ is approached; B($\zeta=0$) must always be zero for a flat radial metric component, where as A($\zeta=0$) decreases rapidly as $\zeta_s$ gets closer for the critical value. This implies that $g_{00}$ approaches 0 the closer to the center of the soliton.

![A values](images/v1_1500_20/A_vs_z.png)
![B values](images/v1_1500_20/B_vs_z.png)

###### A and B values against $\zeta$ for six different $\zeta_s$ parameters. Taken with version 1 code (Seth) with 1500 intervals and $\zeta_{max}=20$.

### 2.2. $\epsilon$ and $A_0$ values vs. $\zeta_s$

The two values for a specific $\zeta_s$ that give the most information about the system is $\epsilon$ for the rescaled energy and $A_0$ for what the time metric component looks like at the center, and consequently determines what the rest of the metric looks like.

After sampling several $\zeta_s$ values from 0 to 0.7427, a trend can be seen with both $\epsilon$ and $A_0$:

![epsilon vs zeta_s](images/v1_1500_20/eps_vs_zs_1500_20_v1.png)
![epsilon vs zeta_s zoomed](images/v1_1500_20/eps_vs_zs_zoomed.png)
![A0 vs zeta_s](images/v1_1500_20/A0_vs_zs_1500_20_v1.png)
![A0 vs zeta_s zoomed](images/v1_1500_20/A0_vs_zs_zoomed.png)

###### Regular and zoomed in (close to $\zeta_{crit}$) graphs from version 1 code (Seth).

![epsilon vs zeta_s](images/v2_1500_20/eps_vs_zs.png)
![epsilon vs zeta_s zoomed](images/v2_1500_20/eps_vs_zs_zoomed.png)
![A0 vs zeta_s](images/v2_1500_20/A0_vs_zs.png)
![A0 vs zeta_s zoomed](images/v2_1500_20/A0_vs_zs_zoomed.png)

###### Regular and zoomed in (close to $\zeta_{crit}$) graphs from version 2 code (Xavier).

An asympotic/exponential curve apprears past $\zeta_s=0.7$ where values rapidly decrease and computational overflow errors occur past $\zeta_s \approx 0.7427$. It becomes clear that the current expression for the metric components ($g_{00}=e^{2A}$ and $g_{rr}=e^{2B}$) breaks down at that area and a new formulation of the metric needs to be considered. 

### 2.2. Schwarchild mass and expectation values

### 2.3. Data modeling for critical $\zeta_s$

## 3. Conclusions and Future Work